# ReadMe
This file is intended to streamline the process of analyzing the data resulting from running the experiments in "falsifier.py" and "mult_obj_falsifier.py"

# Imports - TO DO CONDUCT AN ANALYSIS LOL

In [31]:
import pandas as pd
import os
from pathlib import Path
import re
import numpy as np
import matplotlib.pyplot as plt
import math
from natsort import os_sorted
from sklearn.decomposition import PCA, FactorAnalysis
from sklearn.preprocessing import StandardScaler


# Classify File Instances

In [4]:
def generate_velocity_csv(folder): #TODO - Either fix verifia for multi-obj or modify for that case
    """
        Builds csv containing velocity data for each simulation using the
        corresponding order of the params file returned by verifAI
    """

    ce_reg = r'.*\d_cex.*'
    nc_reg = r'.*no_cex.*'
    att_reg = r".*attacker_crash.*"


    non_ce_reg = [nc_reg,att_reg]
    

    start = os.getcwd()
    v1,v2,v3,v4 = [],[],[],[]

    files = os_sorted(os.listdir(folder))

    os.chdir(folder)
    ce_count = 0
    for file in files: #match counter-examples first
        found = re.match(ce_reg,  file)
        if found:
            file_data = pd.read_csv(file)
            v1.append(np.array(file_data["v1"]))
            v2.append(np.array(file_data["v2"]))
            v3.append(np.array(file_data["v3"]))
            v4.append(np.array(file_data["v4"]))
            ce_count+= 1

    
    for file in files: #match non counter examples second
        for reg in non_ce_reg:
            found = re.match(reg, file)
            if found:
                file_data = pd.read_csv(file)
                v1.append(np.array(file_data["v1"]))
                v2.append(np.array(file_data["v2"]))
                v3.append(np.array(file_data["v3"]))
                v4.append(np.array(file_data["v4"]))
                break

    os.chdir(start)

    return [v1,v2,v3,v4], ce_count


In [52]:

def read_folder(folder,sampler_file, attack_type):    
    """
    Data prepocessing: returns dataframe with test case specific features
    """

    velocity_data,ce_count = generate_velocity_csv(folder)


    simulation_params_df = pd.read_csv(sampler_file)

    if attack_type == "int":
        return(preprocess_intermittent(simulation_params_df,velocity_data)),ce_count
    if attack_type == "per":
        return(preprocess_persistent(simulation_params_df,velocity_data)),ce_count
    else:
        print("Error classifying attack type")


def preprocess_intermittent(sampler_data,velocities_df,):
    """
    Isolates relative columns and generates some new features for analysis

    Side Effect: Generates velocity graphs over time with attack indicators
    """
    intermittent_cols = ["Unnamed: 0","point.params.random_thr_br1","point.params.random_thr_br2","point.params.inter_vehicle_distance", "point.params.random_t1", "point.params.random_t2","point.params.random_t3","point.params.random_t4","rho"]
    sampler_data =sampler_data[intermittent_cols]
    col_names = {
        "Unnamed: 0" : "idx",
        "point.params.random_thr_br1": "random_thr_br1",
        "point.params.random_thr_br2" : "random_thr_br2",
        "point.params.random_t1" : "random_t1",
        "point.params.random_t2" : "random_t2",
        "point.params.random_t3" : "random_t3",
        "point.params.random_t4" : "random_t4",
        "point.params.inter_vehicle_distance" : "inter_vehicle_distance",
        "rho" : "rho"
    }
    sampler_data = sampler_data.copy()
    sampler_data.rename(columns=col_names, inplace=True)
    output = get_attack_spacing_intermittent(sampler_data)
    output = velocity_at_attack_intermittent(sampler_data,velocities_df)
    

    return output

def preprocess_persistent(sampler_data,velocities_df):
    """
    Isolate relative columns and generates new featurs

    Side Effects: Generates velocity graphs over time with attack indicators
    """
    persistent_cols = [ "Unnamed: 0","point.params.amplitude_acc", "point.params.attack_time", "point.params.frequency", "point.params.inter_vehicle_distance", "rho"]
    sampler_data = sampler_data[persistent_cols]
    col_names = {
        "Unnamed: 0" : "idx",
        "point.params.amplitude_acc": "amplitude_acc",
        "point.params.attack_time" : "attack_time",
        "point.params.frequency" : "frequency",
        "point.params.inter_vehicle_distance" : "inter_vehicle_distance",
        "rho" : "rho"
    }
    sampler_data = sampler_data.copy()
    sampler_data.rename(columns=col_names, inplace=True)
    output = velocity_at_attack_persistent(velocities_df,sampler_data,True)
    return output




# Helper Functions

In [83]:
def get_attack_spacing_intermittent(intermittent_df): # Calculate distance between each attack and add to Dataframe
    attack_spacing = []
    """
        Calcluates spacing between attacks and appends to given dataframe
    
    """

    for simulation_instance in range(len(intermittent_df)):
        diff_1 = intermittent_df["random_t2"][simulation_instance] - intermittent_df["random_t1"][simulation_instance] # t2 > t1
        diff_2 = intermittent_df["random_t3"][simulation_instance] - intermittent_df["random_t2"][simulation_instance]
        diff_3 = intermittent_df["random_t4"][simulation_instance] - intermittent_df["random_t3"][simulation_instance]

        sim_gaps = np.array([diff_1,diff_2,diff_3])
        attack_spacing.append(sim_gaps)

    attack_spacing = np.array(attack_spacing)
    col_names = ["t1_t2", "t2_t3", "t3_t4"]
    time_df = pd.DataFrame(attack_spacing, columns=col_names)

    return(pd.concat([intermittent_df, time_df],axis=1))


def velocity_at_attack_intermittent(intermittent_df,velocities_df): # Collect the attackers speed at each attack and append to Dataframe
    """
    Calculates velocity at attack times and appends to dataframe
    """
    times = intermittent_df[["random_t1", "random_t2", "random_t3","random_t4"]]

    attk_vel = []
    for sim in range(len(times)):
        sim_vel = []
        sim_length = len(velocities_df[0][sim])
        for col in times.columns:
            idx = math.ceil(times[col][sim]) * 10
            if idx < sim_length:
                sim_vel.append(velocities_df[0][sim][idx])
            else:
                sim_vel.append(0)
        attk_vel.append(sim_vel)
        
    col_names = ["t1_velocity", "t2_velocity", "t3_velocity", "t4_velocity"]
    attk_vel = np.array(attk_vel)
    final_df = pd.DataFrame(attk_vel, columns=col_names)

    if display:
        generate_intermittent_attack_graphs(velocities_df,times,intermittent_df)

    return (pd.concat([intermittent_df,final_df],axis=1))


def velocity_at_attack_persistent(velocities_df, persistent_df, display):
    """
    Calcluates velocity at attack time (initial), and average velocity for the remainder of the attack
    """
    attack_times= persistent_df["attack_time"]


    velocity_at_attack = []  # construct custom features
    avg_speed_after_attck = []

    for sim_instance in range(len(attack_times)):
        attack_time = math.ceil(attack_times[sim_instance] * 10) # account for timestep length
        velocity_at_attack.append(velocities_df[0][sim_instance][attack_time]) # velocity at attack time
        avg_speed_after_attck.append(np.average(velocities_df[0][sim_instance][attack_time:])) # collect that average speed after attack has occured

    velocity_at_attack = np.array(velocity_at_attack)
    avg_speed_after_attck = np.array(avg_speed_after_attck)
    col_names = [["pre_attack_velocity"],["attack_velocity_average"]]
    temp_df = pd.DataFrame(velocity_at_attack, columns=col_names[0])
    temp_df2 = pd.DataFrame(avg_speed_after_attck,columns=col_names[1]) 

    if display:
        generate_persistent_attack_graphs(velocities_df, persistent_df["frequency"],persistent_df["amplitude_acc"],persistent_df["attack_time"])
        


    return(pd.concat([persistent_df,temp_df,temp_df2],axis=1))                                    


In [ ]:
def distinguish_events(features_df, counter_ex_totals): # Todo improve or build upon to gather attacker_crash instanaces
    ce_df = features_df[:counter_ex_totals]
    non_ce_df = features_df[counter_ex_totals:]

    labels = np.concatenate((np.full(counter_ex_totals,1), np.full(len(features_df)- counter_ex_totals,0)))

    return ce_df, non_ce_df, labels

# Visualization Tools

In [78]:
def generate_persistent_attack_graphs(sim_velocities,att_freq,att_amplitude,times): # okay we gonna have to fix this up later
    """
    Graph Velocity over time w/ displayed amplitude and frequency   
    """
    for i in range(3):
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))  # Two side-by-side plots

        x = range(len(sim_velocities[0][i]))

        # First subplot - Attacker vs Follower 1
        axes[0].plot(x, sim_velocities[0][i], label="Attacker")
        axes[0].plot(x, sim_velocities[1][i], linestyle="--", color="purple", label="Follower 1")

        axes[0].set_title(f"Velocity Over Time\nAttack Freq: {int(att_freq[i])}, Amp: {att_amplitude[i]:.2g}")
        axes[0].set_xlabel(f"Timesteps (.1 secs)")
        axes[0].set_ylabel("Velocity (m/s)")
        axes[0].legend()

        attack_time = math.ceil(times[i] * 10)
        axes[0].axvline(x=attack_time, color='red', alpha=0.7, label="Attack Time")

        # Second subplot - Followers 1, 2, and 3
        x_followers = range(len(sim_velocities[1][i]))
        axes[1].plot(x_followers, sim_velocities[1][i], linestyle="--", color="green", label="Follower 1")
        axes[1].plot(x_followers, sim_velocities[2][i], linestyle="--", color="blue", label="Follower 2")
        axes[1].plot(x_followers, sim_velocities[3][i], linestyle="--", color="purple", label="Follower 3")

        axes[1].set_title(f"Follower Velocities Over Time")
        axes[1].set_xlabel("Timesteps (.1 secs)")
        axes[1].set_ylabel("Velocity (m/s)")
        axes[1].legend()

        axes[1].axvline(x=attack_time, color='red', alpha=0.7, label="Attack Time")

        plt.tight_layout()  # Adjust layout for better spacing
        plt.show()

def generate_intermittent_attack_graphs(velocities_list,times,intermittent_df):
    """
    Graph velocity over time w/ attack markers displaying attack start"
    
    """

    for i in range(0, 3):
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))  # Create a figure with 2 subplots side by side

        # First subplot - Attacker velocity
        x = range(len(velocities_list[0][i]))
        axes[0].plot(x, velocities_list[0][i], label="Attacker")
        axes[0].plot(x, velocities_list[1][i], linestyle="--", color="purple", label="Follower 1")

        axes[0].set_title("Attacker Velocity (m/s) with attack markers")
        axes[0].set_xlabel("Timesteps (.1 secs)")
        axes[0].set_ylabel("Velocity (m/s)")
      
        first = True
        for j in times.columns:
            attack_time = math.ceil(times[j][i] * 10)
            label = "Attack" if first else "_nolegend_"
            axes[0].axvline(x=attack_time, color='red', alpha=0.7, label=label )
            first = False


        axes[0].legend()

        # Second subplot - Followers velocity
        x = range(len(velocities_list[2][i]))
        axes[1].plot(x, velocities_list[1][i], linestyle="--", alpha=.7,  color="green", label="Follower 1")
        axes[1].plot(x, velocities_list[2][i], linestyle="--", alpha=.7, color="blue", label="Follower 2")
        axes[1].plot(x, velocities_list[3][i], linestyle="--", alpha=.7, color="purple", label="Follower 3")

        axes[1].set_title("Followers Velocity (m/s) over time with attack markers")
        axes[1].set_xlabel("Timesteps (.1 secs)")
        axes[1].set_ylabel("Velocity (m/s)")

        first = True
        for j in times.columns:
            attack_time = math.ceil(times[j][i] * 10)
            label = "Attack" if first else "_nolegend_"
            axes[1].axvline(x=attack_time, color='red', alpha=0.7, label=label )
            first = False

        axes[1].legend()

        # Determine shared limits for x-axis and y-axis
        x_min = 0
        x_max = max(len(velocities_list[0][i]), len(velocities_list[2][i]))

        y_min = min(
            min(velocities_list[0][i]), min(velocities_list[1][i]),
            min(velocities_list[2][i]), min(velocities_list[3][i])
        )
        y_max = max(
            max(velocities_list[0][i]), max(velocities_list[1][i]),
            max(velocities_list[2][i]), max(velocities_list[3][i])
        )

        # Apply limits to both subplots
        axes[0].set_xlim(x_min, x_max)
        axes[1].set_xlim(x_min, x_max)
        axes[0].set_ylim(y_min, y_max)
        axes[1].set_ylim(y_min, y_max)


        plt.tight_layout()  # Adjust layout to fit both plots properly
        plt.show()


# Cofactor Analysis

In [ ]:
def cofactor_analysis(ce, non_ce):
    feature_names = ce.columns   
    X_ce = np.array(ce)
    X_non_ce = np.array(non_ce)
    
    X_ce_scaled = StandardScaler().fit_transform(X_ce)
    X_non_ce_scaled = StandardScaler().fit_transform(X_non_ce)
    
    corr_matrix_ce = np.corrcoef(X_ce_scaled.T)
    corr_matrix_non_ce = np.corrcoef(X_non_ce_scaled.T)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 7))
    
    im1 = ax1.imshow(corr_matrix_ce, cmap="RdBu_r", vmin=-1, vmax=1)
    ax1.set_xticks(range(len(feature_names)))
    ax1.set_xticklabels(feature_names)
    ax1.set_yticks(range(len(feature_names)))
    ax1.set_yticklabels(feature_names)
    ax1.set_title("Correlation Matrix for Counter-Examples")
    plt.setp(ax1.get_xticklabels(), rotation=90, ha='right')  # Rotate x labels for readability
    
    im2 = ax2.imshow(corr_matrix_non_ce, cmap="RdBu_r", vmin=-1, vmax=1)
    ax2.set_xticks(range(len(feature_names)))
    ax2.set_xticklabels(feature_names)
    ax2.set_yticks(range(len(feature_names)))
    ax2.set_yticklabels(feature_names)
    ax2.set_title("Correlation Matrix for No Counter-Examples")
    plt.setp(ax2.get_xticklabels(), rotation=90, ha='right')  # Rotate x labels for readability
    
    fig.colorbar(im1, ax=ax1, fraction=0.046, pad=0.04)
    fig.colorbar(im2, ax=ax2, fraction=0.046, pad=0.04)
    
    plt.tight_layout()
    plt.show()

In [ ]:
ce,non_ce, labels= distinguish_events(processed_data,ce_count1)
ce2, non_ce2, labels = distinguish_events(pd_2, ce_count2)

cofactor_analysis(ce,non_ce,labels)
cofactor_analysis(ce2,non_ce2,labels)

# Test

In [107]:
file_path = "../outputs/persistent_ce/persistent_attack_ce_ce.csv"
dir_path = "../outputs/persistent_ce"

file_path2 = "../outputs/intermittent_ce/intermittent_attack_ce_ce.csv"
dir_path2 = "../outputs/intermittent_ce"


In [ ]:
processed_data, ce_count1 = read_folder(dir_path, file_path,"per")
pd_2, ce_count2  = read_folder(dir_path2, file_path2, "int")

In [153]:
pd_2 = pd_2.drop(columns=["idx","rho"])
processed_data = processed_data.drop(columns=["idx","rho"])

# PCA Analysis

In [ ]:
pca = PCA(n_components="mle")


# Curr work

Persistent_attack uses frequency to define how often the attack is applied. What are some interesting feature we can extract from this? 
 Total attacks?
 Time between attacks?
 Velocity at attack time would still be interesting just a bit more complex to extract.


Likely need to rerun data as there was a useless variable EGO_BRAKING_THESHOLD being included in the search space.. I am not sure how much of an effect this would have had but can always re run and see what happens

In [ ]:
file_path = "../outputs/persistent_ce/persistent_attack_ce.csv"
dir_path = "../outputs/persistent_ce"

simulation_params_df = pd.read_csv("../outputs/persistent_ce/persistent_attack_ce.csv")

In [ ]:
persistent_cols = [ "point.params.amplitude_acc", "point.params.attack_time", "point.params.frequency", "point.params.inter_vehicle_distance", "rho"]
intemittent_cols = ["point.params.random_thr_br1","point.params.random_thr_br2","point.params.inter_vehicle_distance", "point.params.random_t1", "point.params.random_t2","point.params.random_t3","point.params.random_t4","point.params.random_thr_br1","rho"]

In [ ]:
evaluation_simulation_params_df = simulation_params_df[persistent_cols]
print(len(evaluation_simulation_params_df))

500


In [ ]:
files = os.listdir(dir_path)
classified_files = classify_simulations(files)

In [ ]:
evaluation_simulation_params_df.describe()

,point.params.amplitude_acc,point.params.attack_time,point.params.frequency,point.params.inter_vehicle_distance,rho
count,500.000000,500.000000,500.000000,500.000000,500.000000
mean,0.341717,6.507021,6.109624,11.331299,2.282775
std,0.274452,1.975794,2.955776,4.929331,4.627224
min,0.001723,0.036584,0.057499,7.008833,-0.273221
25%,0.124351,6.188735,3.267220,8.425303,0.066527
50%,0.265883,6.950120,6.355811,9.749395,0.242624
75%,0.534049,7.575427,9.023248,11.140633,1.478580
max,0.994191,9.980661,9.993043,29.065113,23.108299


In [ ]:
vel_path = "../outputs/persistent_ce/persistent_attack.scenic500_velocities.csv"
vel_df = pd.read_csv(vel_path)

# Constructing features for the persistent Attack

In [ ]:
attack_freq = evaluation_simulation_params_df["point.params.frequency"]
attack_amplitude = evaluation_simulation_params_df["point.params.amplitude_acc"]


In [ ]:
generate_persistent_attack_graphs(simulation_velocities_arr,attack_freq,attack_amplitude)

# Isolate different crash instances